# machine_learning.ipynb

## 1. Introducción

En este Notebook, vamos a:
1. Cargar los distintos datasets de entrenamiento y test (con/sin outliers, normalizados o escalados).
2. Entrenar un modelo de **Regresión Logística** en cada uno de ellos y medir su desempeño.
3. Escoger el mejor dataset con base en las métricas.
4. Realizar una **búsqueda de hiperparámetros** (*GridSearchCV*) para optimizar el modelo ganador.
5. Guardar o mostrar los resultados finales.

Este enfoque modular permite:
- Comparar rápidamente múltiples versiones de preprocesamiento (outliers, escalados, etc.).
- Reutilizar el mismo pipeline en proyectos futuros con datasets distintos.


In [ ]:

# 1. Importación de librerías

import os
import pandas as pd
import numpy as np

# Para modelado
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Ignorar warnings (opcional)
import warnings
warnings.filterwarnings("ignore")



## 2. Carga de datos

En este apartado, definimos las rutas de los archivos `.xlsx` que contienen los distintos **X_train** y **X_test** (con outliers, sin outliers, normalizados, escalados, etc.), así como los vectores `y_train` y `y_test`. 


In [ ]:

# 2. Carga de datos

# Ruta base a la carpeta processed
BASE_PATH = "/workspaces/ejercicio_regresion_logistica/data/processed"

# Archivos disponibles según tu carpeta
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_normalizado.xlsx",
    "X_train_sin_outliers_normnormalizado.xlsx",
    "X_train_con_outliers_escalado.xlsx",
    "X_train_sin_outliers_escalado.xlsx"
]

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_normalizado.xlsx",
    "X_test_sin_outliers_normalizado.xlsx",
    "X_test_con_outliers_escalado.xlsx",
    "X_test_sin_outliers_escalado.xlsx"
]

# Carga de datasets
TRAIN_DATASETS = [pd.read_excel(os.path.join(BASE_PATH, path)) for path in TRAIN_PATHS]
TEST_DATASETS = [pd.read_excel(os.path.join(BASE_PATH, path)) for path in TEST_PATHS]

# Carga de las variables objetivo
y_train = pd.read_excel(os.path.join(BASE_PATH, "y_train_sel.xlsx")).iloc[:, 0]
y_test = pd.read_excel(os.path.join(BASE_PATH, "y_test_sel.xlsx")).iloc[:, 0]

# Verificación
print("Cantidad de datasets de TRAIN cargados:", len(TRAIN_DATASETS))
print("Cantidad de datasets de TEST cargados:", len(TEST_DATASETS))


Cantidad de datasets de TRAIN cargados: 3
Cantidad de datasets de TEST cargados: 3


In [ ]:

# 3. Entrenamiento de modelos de regresión logística + Evaluación

results = []  # Lista para almacenar métricas

# Entrenar y evaluar modelo por cada dataset
for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    
    model = LogisticRegression(random_state=42, max_iter=1000)  # max_iter por si tarda en converger
    model.fit(dataset, y_train)
    
    # Predicciones
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])
    
    # Métricas
    # train_acc = accuracy_score(y_train, y_pred_train)
    # test_acc = accuracy_score(y_test, y_pred_test)
    
    
    results.append(
        {
            # "dataset": TRAIN_PATHS[index],
            "train": accuracy_score(y_train, y_pred_train),
            "test": accuracy_score(y_test, y_pred_test)
        }
    )

results

# print("\nResultados de accuracy por dataset:")
# for res in results:
#    print(res)


0
1
2


[{'train': 0.9083459787556905, 'test': 0.9104151493080845},
 {'train': 0.908103186646434, 'test': 0.9113862588006798},
 {'train': 0.9004552352048558, 'test': 0.9047098810390871}]

In [ ]:

# 4. Optimización con GridSearchCV del mejor modelo

# Suponiendo que el dataset seleccionado fue el tercero (por ejemplo, con variables seleccionadas)
best_dataset_index = 2  # Esto lo puedes ajustar según los resultados del paso anterior

# Definimos los hiperparámetros para buscar
hyperparams = {
    "penalty": ["l1", "l2", "elasticnet", None],
    "tol": [0.0001, 0.001, 0.1],
    "fit_intercept": [True, False],
    "C": [0.1, 1, 10]  # Regularización
}

# Crear modelo y GridSearch
model = LogisticRegression(random_state=42, solver='saga', max_iter=1000)  # 'saga' soporta l1, l2 y elasticnet
grid = GridSearchCV(model, hyperparams, scoring="accuracy", cv=5)

# Entrenar GridSearch
print("\nEjecutando GridSearchCV...")
grid.fit(TRAIN_DATASETS[best_dataset_index], y_train)

# Mostrar mejor modelo
print("\nMejor modelo encontrado por GridSearch:")
print(grid.best_estimator_)



Ejecutando GridSearchCV...

Mejor modelo encontrado por GridSearch:
LogisticRegression(C=1, max_iter=1000, random_state=42, solver='saga', tol=0.1)


In [ ]:

# 5. Evaluación final del mejor modelo

final_model = grid.best_estimator_

# Predicciones finales
y_pred_train_final = final_model.predict(TRAIN_DATASETS[best_dataset_index])
y_pred_test_final = final_model.predict(TEST_DATASETS[best_dataset_index])

# Accuracy final
final_train_accuracy = accuracy_score(y_train, y_pred_train_final)
final_test_accuracy = accuracy_score(y_test, y_pred_test_final)

print("\nResultados finales del mejor modelo:")
print(f"Accuracy final en TRAIN: {final_train_accuracy:.4f}")
print(f"Accuracy final en TEST: {final_test_accuracy:.4f}")



Resultados finales del mejor modelo:
Accuracy final en TRAIN: 0.9008
Accuracy final en TEST: 0.9046
